# self-attention

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
### 랜덤 시드 설정
tf.random.set_seed(0)

## 과정 1 : 임베딩 행렬 생성

In [3]:
'''
1. sentence = "I love you"
2. 토큰화 --> [I, love, you]
3. 각 토큰 --> 512 차원의 임베딩 벡터로 변환  --> (3, 512) 임베딩 행렬 생성
4. 표준 정규 분포로부터 랜덤한 실수 샘플링 --> (3, 512) 임베딩 행렬에 해당하는 데이터 생성
'''

## (3, 512) 임베딩 행렬에 해당하는 데이터 생성
embedding_shape = (3, 512)
embedding_matrix = tf.random.normal(shape = embedding_shape)
print(f'임베딩 행렬에 해당하는 데이터 : \n{embedding_matrix}')

print('-'*80)

## positional encoding에 해당하는 데이터 생성

# zero 행렬 생성
zero_matrix = np.zeros(shape = embedding_shape)
print(zero_matrix)
print('-'*80)

# I : 첫번째 위치에 1, love : 두번째 위치에 1, you : 세번째 위치에 1 입력
zero_matrix[0, 0] = 1
zero_matrix[1, 1] = 1
zero_matrix[2, 2] = 1
positional_encoding = zero_matrix[:, :]
print(positional_encoding)
print('-'*80)

## ebedding 행렬 + position encoding
embedding_matrix = embedding_matrix + positional_encoding
print(embedding_matrix)

임베딩 행렬에 해당하는 데이터 : 
[[ 1.5110626   0.42292204 -0.41969493 ... -1.1673577   0.777814
   0.58657396]
 [-0.13087033 -0.4497122   3.3774817  ...  0.2500961  -0.69026154
  -0.8148735 ]
 [ 0.40156764  0.3129424  -0.87114996 ... -0.3053926   0.18731174
  -1.6565207 ]]
--------------------------------------------------------------------------------
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
--------------------------------------------------------------------------------
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
--------------------------------------------------------------------------------
tf.Tensor(
[[ 2.5110626   0.42292204 -0.41969493 ... -1.1673577   0.777814
   0.58657396]
 [-0.13087033  0.55028784  3.3774817  ...  0.2500961  -0.69026154
  -0.8148735 ]
 [ 0.40156764  0.3129424   0.12885004 ... -0.3053926   0.18731174
  -1.6565207 ]], shape=(3, 512), dtype=float32)


## 과정 2 : query, key, value 생성

In [4]:
### 가중치 행렬 생성

## 랜덤 시드 설정
tf.random.set_seed(0)

'''
1. 가중치 행렬 W_q, W_k, W_v의 모양 : (512, 512)
2. 가중치 행렬의 초기 값 --> 랜덤한 실수로 설정
3. 표준 정규 분포로부터 랜덤한 실수 샘플링 --> (512, 512) 가중치 행렬에 해당하는 데이터 W_q, W_k, W_v 생성
'''

# (512, 512*3) 모양의 행렬 생성
weights_shape = (512, 512*3)
W = tf.random.normal(shape=weights_shape)
print(f'가중치 행렬 : \n{W}')

# 가중치 행렬 W --> W_q, W_k, W_v 생성
W_q = W[:, 0:512]
W_k = W[:, 512:512*2]
W_v = W[:, 512*2:]

# 결과 확인하기
print(f'가중치 행렬 W_q의 모양 : {W_q.shape}')
print('-'*80)
print(f'가중치 행렬 W_q : \n{W_q}')
print('-'*80)
print(f'가중치 행렬 W_k의 모양 : {W_k.shape}')
print('-'*80)
print(f'가중치 행렬 W_k : \n{W_k}')
print('-'*80)
print(f'가중치 행렬 W_v의 모양 : {W_v.shape}')
print('-'*80)
print(f'가중치 행렬 W_v : \n{W_v}')

가중치 행렬 : 
[[ 1.5110626   0.42292204 -0.41969493 ... -0.3053926   0.18731174
  -1.6565207 ]
 [ 0.2958256   1.4492193  -0.346698   ...  1.041023   -0.56670994
   0.2789855 ]
 [ 0.6262866  -0.37516117  0.388404   ... -0.6312742   0.9733827
  -0.42035455]
 ...
 [ 1.8720272  -1.2698175  -0.23749031 ...  0.7807198  -2.7204087
   0.05482808]
 [ 0.25407916  0.6127431   1.0087564  ...  1.0627506  -0.6406648
   1.0110184 ]
 [-0.02076815 -1.021457   -0.99233574 ... -1.0839558   0.37213057
  -0.31700885]]
가중치 행렬 W_q의 모양 : (512, 512)
--------------------------------------------------------------------------------
가중치 행렬 W_q : 
[[ 1.5110626   0.42292204 -0.41969493 ... -1.1673577   0.777814
   0.58657396]
 [ 0.2958256   1.4492193  -0.346698   ... -0.74473715 -0.3300144
   0.55726486]
 [ 0.6262866  -0.37516117  0.388404   ...  0.57635653 -2.1872427
  -0.5449532 ]
 ...
 [ 1.8720272  -1.2698175  -0.23749031 ...  0.39929047  1.2053078
   1.2300565 ]
 [ 0.25407916  0.6127431   1.0087564  ... -1.1795905  

In [5]:
'''
### 임베딩 행렬과 가중치 행렬곱 --> 각 단어의 q, k, v 백터 생성
'''

# 임베딩 행렬과 가중치 행렬 곱
qkv = tf.matmul(a = embedding_matrix, b = W)
print(f'생성된 qkv 모양 : {qkv.shape}')
print('-'*80)
print(f'생성된 qkv : \n{qkv}')
print('-'*80)

# 각 단어의 q, k, v 생성 --> qkv 행렬 --> 슬라이싱
q = qkv[:, 0:512]
k = qkv[:, 512:1024]
v = qkv[:, 1024:]

# 결과 확인하기
print(f'q 행렬의 모양 : {q.shape}')
print('-'*80)
print(f'q 행렬 : \n{q}')
print('-'*80)
print(f'k 행렬의 모양 : {k.shape}')
print('-'*80)
print(f'k 행렬 : \n{k}')
print('-'*80)
print(f'v 행렬의 모양 : {v.shape}')
print('-'*80)
print(f'v 행렬 : \n{v}')

생성된 qkv 모양 : (3, 1536)
--------------------------------------------------------------------------------
생성된 qkv : 
[[  6.752496  -34.07061   -19.767971  ... -16.244774   17.998104
   14.833261 ]
 [-21.453941   15.075088   11.459349  ... -10.253195   16.947517
   20.262678 ]
 [  0.6940522   7.215227   45.29867   ...  -3.735396   17.622162
    4.451379 ]]
--------------------------------------------------------------------------------
q 행렬의 모양 : (3, 512)
--------------------------------------------------------------------------------
q 행렬 : 
[[  6.752496  -34.07061   -19.767971  ...   8.092958  -37.391403
   27.132042 ]
 [-21.453941   15.075088   11.459349  ... -32.86045   -44.916115
  -58.86846  ]
 [  0.6940522   7.215227   45.29867   ... -23.705288   -6.4933653
   -3.521988 ]]
--------------------------------------------------------------------------------
k 행렬의 모양 : (3, 512)
--------------------------------------------------------------------------------
k 행렬 : 
[[ 4.2129219e+01  1.78

## 과정 3 : Scaled_dot_product_attention



### attention score

In [6]:
### query 행렬과 key 행렬간의 행렬 곱
"""
1. key 행렬 전치(transpoe) --> (3, 512) --> (512, 3)
2. query 행렬과 전치된 key 행렬 --> 행렬 곱
"""

attention_score = tf.matmul(a = q, b = k, transpose_b = True)
print(attention_score)

tf.Tensor(
[[  1703.1049    -713.31323  -3711.223  ]
 [-21326.793    -5222.2803   20668.867  ]
 [  6377.7256   10644.914     6412.1963 ]], shape=(3, 3), dtype=float32)


### scaling

In [7]:
'''
scaling : attention_score / 제곱근(512)
'''
# 각 단어의 key 벡터 --> 크기 추출
size = k.shape[1]
print(f'key 벡터의 크기 : {size}')
print('-'*80)
print(f'key 벡터의 자료형 : {type(size)}')
print('-'*80)

# key 벡터의 크기(size) --> 숫자의 자료형 --> 자료형 변환
size = tf.cast(size, tf.float32)
# size = tf.constant(size, dtype = tf.float32)
print(f'변환된 size : {size}')
print('-'*80)
print(f'변환된 size 변수의 자료형 확인 : {type(size)}')
print('-'*80)


# scaled_attention_score / 제곱근(512)
scaled_attention_score = attention_score / tf.sqrt(size)
print(f'스케일링을 한 attention_score : \n{scaled_attention_score}')

key 벡터의 크기 : 512
--------------------------------------------------------------------------------
key 벡터의 자료형 : <class 'int'>
--------------------------------------------------------------------------------
변환된 size : 512.0
--------------------------------------------------------------------------------
변환된 size 변수의 자료형 확인 : <class 'tensorflow.python.framework.ops.EagerTensor'>
--------------------------------------------------------------------------------
스케일링을 한 attention_score : 
[[  75.26731  -31.52429 -164.01443]
 [-942.52    -230.79437  913.44354]
 [ 281.8583   470.44318  283.3817 ]]


### attention_weights

In [8]:
'''
### 스케일링 한 attention_score --> softmax 함수 --> tf.nn.softmax
'''
attention_weights = tf.nn.softmax(scaled_attention_score)
print(f'attention_weights : \n{attention_weights}')

attention_weights : 
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [9]:
np.sqrt(3)

1.7320508075688772

### attention_output : contextual embedding vectors

In [14]:
# 학습의 과정에 의해서 업데이트가 완료된 가중치(예시)
attention_weights = [[0.9, 0.03, 0.07], [0.05, 0.85, 0.1], [0.06, 0.04, 0.9]]

# python list 자료형 --> tf.Tensor 배열로 변환
# attention_weights = tf.convert_to_tensor(attention_weights)
# print(f'자료형 변환 후 결과 : \n{attention_weights})

# print('-'*80)

# 업데이트가 완료된 가중치와 각 단어의 value 행렬간의 행렬 곱 --> 결과물 --> 각 단어의 최종 임베딩 벡터 생성(예시)
output = tf.matmul(a = attention_weights, b = v)
print(f'각 단어의 최종 임베딩 벡터 : \n{output}')

각 단어의 최종 임베딩 벡터 : 
[[ 10.488299  -13.140542   31.937096  ... -15.18937    17.94027
   14.269412 ]
 [-29.80416    -5.3819036 -34.029533  ...  -9.900994   17.06751
   18.410076 ]
 [-12.811751   -8.6698    -16.844946  ...  -4.7466702  17.617733
    5.7067437]]


In [15]:
print(output.shape)

(3, 512)
